# 14-xml输出解析器

该输出解析器允许用户以流行的 XML 格式从 LLM 获取结果。

请记住，大型语言模型是有漏洞的抽象！您必须使用具有足够容量的 LLM 来生成格式正确的 XML。

! pip install defusedxml

In [1]:
# 定义大模型
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)

In [2]:
# 定义输出解析器，并定义响应数据格式
from langchain_core.output_parsers import XMLOutputParser

parser = XMLOutputParser()
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


In [3]:
format_instructions = """请直接返回XML格式的字符串，不要返回任何代码。
必须严格按照以下XML结构返回（不要包含任何其他内容）：

<xml>
    <name>姓名</name>
    <age>年龄</age>
    <gender>性别</gender>
    <film_list>
        <movie>电影1</movie>
        <movie>电影2</movie>
    </film_list>
</xml>
"""

In [4]:
# 定义提示词模板
from langchain_core.prompts import PromptTemplate
template = """请根据以下演员名字，生成一个包含其基本信息的XML格式响应。
演员：{name}

要求：
1. 必须使用XML格式
2. 信息要真实准确
3. 电影列表至少包含5部代表作

{format_instructions}"""

prompt = PromptTemplate(
    template=template,
    input_variables=["name"],
    partial_variables={"format_instructions": format_instructions}
)

print(prompt.format(name="成龙"))

请根据以下演员名字，生成一个包含其基本信息的XML格式响应。
演员：成龙

要求：
1. 必须使用XML格式
2. 信息要真实准确
3. 电影列表至少包含5部代表作

请直接返回XML格式的字符串，不要返回任何代码。
必须严格按照以下XML结构返回（不要包含任何其他内容）：

<xml>
    <name>姓名</name>
    <age>年龄</age>
    <gender>性别</gender>
    <film_list>
        <movie>电影1</movie>
        <movie>电影2</movie>
    </film_list>
</xml>



In [6]:
# 创建链
chain = prompt | llm | parser

# 执行链
result = chain.invoke({"name": "刘亦菲"})
print(result)


{'xml': [{'name': '刘亦菲'}, {'age': '34'}, {'gender': '女'}, {'film_list': [{'movie': '金粉世家'}, {'movie': '天龙八部'}, {'movie': '神雕侠侣'}, {'movie': '仙剑奇侠传'}, {'movie': '功夫之王'}]}]}
